# Read in in LA-COMPASS 2D data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from pathlib import Path

from pylacompass import read_data, read_hdf5_file, twod_plot, read_torqfile, convert_to_cgs, get_snapshot_numbers

au = c.au.cgs.value
yr = (u.yr).to(u.s)

%matplotlib inline

In [ ]:
plt.style.use({'figure.dpi': 150})
figopts = dict(facecolor='none', transparent=True, dpi=300, bbox_inches='tight')

## Reading data

Set filename of the `hdf5` file

In [ ]:
dirname = '/scratch/users/birnstiel/vortex'
n = 200
fname = str(Path(dirname)/'data_yaping.hdf5')

This reads in the specified directory from a sub-directory `bin_data`. The simulation parameters are parsed from the file given as `inputfile`. If keyword `fname` is given, we store the result in a hdf5 file with that name.

In [ ]:
d = read_data(directory=dirname, inputfile='planet2D.input.new', n=n, fname=fname, gridfile=f'{dirname}/log_grid.dat')

In [ ]:
get_snapshot_numbers(fname)

Once the hdf5 file exists, we can also read from it. By default that just opens the file and the data is read when needed. If everything is to be read at once into memory, pass the `lowmem=False` keyword.

In [ ]:
d, f5  = read_hdf5_file(fname, n=n, lowmem=False)

Convert to cgs units

In [ ]:
dcgs = convert_to_cgs(d)

Also read the torq file

In [ ]:
dt = read_torqfile(d, Path(dirname) / 'torq1d.dat')

# Find largest particle size

**Note:** some of the methods used here require `d.sigma_d` to be a numpy array. If you used the `lowmem` keyword of `read_hdf5_file`, then you need to replace `d.sigma_d` with a 

In [ ]:
# define a boolean condition to determine whether the bin is considered filled or empty

cond = d.sigma_d / (np.sum(d.sigma_d,-1)[:, :, None] + 1e-45) > 1e-10

# find the last filled index (argmax finds the first maximum, so we invert the array)

max_index = cond.shape[-1] - cond[:, :, ::-1].argmax(-1) - 1

a_max = d.a[max_index]

## Prepare plotting

set regions for the zoom by calculating where the planet should be

In [ ]:
r_pl = dcgs.params['r0_pl'] * dcgs.r_unit  # planet semi-major axis
omega_pl = np.interp(r_pl, dcgs.x, dcgs.v_k) / r_pl  # planet angular velocity

phi = omega_pl * dcgs.time
x = r_pl * np.cos(phi) / au
y = r_pl * np.sin(phi) / au

# for some reason this is not working will set x and y by hand for now

x = 25
y = 28
r = np.sqrt(x**2+y**2)
dr = 5
phi = np.arctan2(y, x)

ix0 = dcgs.x.searchsorted((r - 2 * dr) * au)
ix1 = dcgs.x.searchsorted((r + 2 * dr) * au)

iy0 = dcgs.y.searchsorted(phi - 2 * dr / r)
iy1 = dcgs.y.searchsorted(phi + 2 * dr / r)

region = [x - dr, x + dr, y - dr, y + dr]
zoom = 5

In [ ]:
def fixplot(f1, title):
    "Function to fine tune most plots in this notebook"
    ax = f1.get_axes()[0]
    cb = f1.get_axes()[-1]
    #ax.set_axis_off()
    ax.set_xlabel('x [au]')
    ax.set_ylabel('y [au]')
    cb.set_title(title)
    
    r = 100
    ax.set_xlim(-r, r)
    ax.set_ylim(-r, r)

## Gas Surface Density

In [ ]:
f1 = twod_plot(dcgs, dcgs.sigma_g, zoom=zoom, region=region, r_unit=au, cb_orientation='horizontal')
fixplot(f1, r'$\log \Sigma_\mathrm{gas}$ [g cm$^{-2}$]')
ax = f1.get_axes()[0]
f1.savefig('xy_gas.pdf', **figopts)

## Total Dust Surface Density

In [ ]:
sig_d_t = dcgs.sigma_d.sum(-1)

In [ ]:
f1 = twod_plot(dcgs, sig_d_t, zoom=zoom, region=region, r_unit=au, cb_orientation='horizontal')
fixplot(f1, r'$\log \Sigma_\mathrm{dust}$ [g cm$^{-2}$]')
f1.savefig('xy_dust.pdf', **figopts)

In [ ]:
# plot dust surface density
f, ax = plt.subplots(figsize=(5,5))
ax.pcolormesh(dcgs.xy1[iy0:iy1,ix0:ix1] / au, dcgs.xy2[iy0:iy1,ix0:ix1] / au, np.log10(sig_d_t[ix0:ix1,iy0:iy1].T+1e-5),rasterized=True)
ax.set_xlim([x - dr, x + dr])
ax.set_ylim([y - dr, y + dr])
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])
f.savefig('xy_dust_zoom.pdf', **figopts)

### Quiver Plot

In [ ]:
def get_vxy(d):
    """
    Calculate the gas x,y velocity minus the Keplerian velocity (of every radius)
    """
    PHI = (np.arctan2(d.xy2,d.xy1)+2*np.pi)%(2*np.pi)
    vr  = d.vr_g.T
    vp  = d.vp_g.T
    
    vk = dcgs.v_k[None,:]

    vx  = vr * np.cos(PHI) - (vp-vk) * np.sin(PHI)
    vy  = vr * np.sin(PHI) + (vp-vk) * np.cos(PHI)
    
    return vx.T, vy.T

vx, vy = get_vxy(dcgs)


# plot velocity
st = 8
f, ax = plt.subplots()
ax.pcolormesh(dcgs.xy1[iy0:iy1,ix0:ix1] / au, dcgs.xy2[iy0:iy1,ix0:ix1] / au, np.log10(sig_d_t[ix0:ix1,iy0:iy1].T+1e-5),rasterized=True)
ax.quiver(dcgs.xy1[iy0:iy1:st,ix0:ix1:st] / au, dcgs.xy2[iy0:iy1:st,ix0:ix1:st] / au,
          vx[ix0:ix1:st,iy0:iy1:st].T / au, vy[ix0:ix1:st,iy0:iy1:st].T / au,
         angles='xy', scale_units='xy', scale=None)
#              )
ax.set_xlim([x - dr, x + dr])
ax.set_ylim([y - dr, y + dr])
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('equal')
f.savefig('xy_dust_zoom_quiver.pdf', **figopts)

### Plot optical depth

In [ ]:
import dsharp_opac as op
from scipy.interpolate import interp2d

lam0 = 0.125

# load opacity

with np.load(op.get_datafile('default_opacities.npz')) as f:
    a     = f['a']
    lam   = f['lam']
    k_abs = f['k_abs']
    rho_s = f['rho_s']

# interpolate on our size grid

f_opac = interp2d(np.log10(lam), np.log10(a), np.log10(k_abs))
opac = 10.**f_opac(np.log10(lam0), np.log10(dcgs.a))

# calculate optical depth

tau = (dcgs.sigma_d * opac[:,0]).sum(-1)

In [ ]:
# plot optical depth
f, ax = plt.subplots(figsize=(6,6))
ax.set_aspect('equal')
cc=ax.pcolormesh(dcgs.xy1[iy0:iy1,ix0:ix1] / au, dcgs.xy2[iy0:iy1,ix0:ix1] / au, np.log10(tau[ix0:ix1,iy0:iy1].T+1e-5),
              rasterized=True,vmin=-2,vmax=0)
p = ax.get_position()
cax = f.add_axes([p.x0,p.y1,p.width,p.height/20])
cb=plt.colorbar(mappable=cc,cax=cax,extend='both',orientation='horizontal')
cb.set_label(r'$\tau$' + f' at {10*lam0:2.0g} mm')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
ax.set_xlim([x - dr, x + dr])
ax.set_ylim([y - dr, y + dr])
ax.set_xticks([])
ax.set_yticks([])
f.savefig('tau.pdf', **figopts)

## Dust surface density of particles $\geq X$ 

In [ ]:
X  = 0.3 # minimum particle size in cm
ia = d.a.searchsorted(X)
f1 = twod_plot(dcgs, dcgs.sigma_d[:, :, ia:].sum(-1), zoom=zoom, region=region, r_unit=au, cb_orientation='horizontal')
fixplot(f1, f'$\log \Sigma_\mathrm{{dust}}$ (a > {10 * X:.1g} mm) [g cm$^{{-2}}$]')
f1.savefig('xy_mm.pdf', **figopts)

### Plot $a_\mathrm{max}$

In [ ]:
# plot a_max
f, ax = plt.subplots()
cc = ax.pcolormesh(dcgs.xy1[iy0:iy1,ix0:ix1] / au, dcgs.xy2[iy0:iy1,ix0:ix1] / au, np.log10(a_max[ix0:ix1,iy0:iy1].T),
              rasterized=True,
              vmin=-2, vmax=np.log10(0.4),
              #edgecolor='0.5', linewidth=.1
              )
ax.set_xlim([x - dr, x + dr])
ax.set_ylim([y - dr, y + dr])
ax.set_aspect('equal')
plt.colorbar(cc);

In [ ]:
f3 = twod_plot(dcgs, a_max, zoom=20, region=region, r_unit=au, pos='r', cb_orientation='horizontal')
fixplot(f3, 'max. particle size [cm]')
f3.savefig('a_max.pdf', **figopts)

### Plot 1D (azimuthally averaged) gas and total dust

In [ ]:
f, ax = plt.subplots()
ax.loglog(dcgs.x / au, dcgs.sigma_g[:, :].mean(1), label='gas')
ax.loglog(dcgs.x / au, dcgs.sigma_d[:, :, :].sum(-1).mean(-1), label='dust')
for phi in [0, 90, 180, 270]:
    iphi = dcgs.y.searchsorted(phi/180*np.pi)
    ax.loglog(dcgs.x / au, dcgs.sigma_d[:, iphi, :].sum(-1), label=f'radial slice at {dcgs.y[iphi]/np.pi:.2g} $\pi$')
ax.set_xlabel('radius')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.legend(fontsize='small')
#ax.set_ylim(1e-4, 1e3)
f.savefig('sigma_avg.pdf')

### Plot azimuthally averaged size distribution

In [ ]:
r  = 1.5
ir = d.x[()].searchsorted(r)

f, ax = plt.subplots()
ax.loglog(d.a, d.sigma_d[ir, :, :].mean(0))
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
ax.set_ylim(1e-10, 1e-1)
f.savefig('size_at_{:.2g}.pdf'.format(r))

### Check unit conversion and calculate Keplerian velocity

Plot the $\Sigma_\mathrm{gas}$ to check conversion

In [ ]:
f, ax = plt.subplots()
ax.loglog(dcgs.x / au, dcgs.sigma_g.mean(-1), label='cgs data from binary')
ax.loglog(dt.r / au, dt.sigma_g[dcgs.n], '--', label='from torq1d.dat')
ax.set_xlabel('$r$ [au]')
ax.set_ylabel('$\Sigma_\mathrm{gas}$ [g cm$^{-2}$]')
ax.legend();

### Plot the azimuthal gas velocity deviation

Plot the radial profile to check conversion

In [ ]:
f, ax = plt.subplots(2, 1, sharex=True, figsize=(6, 6))
ax[0].loglog(dcgs.x / au, (dcgs.v_k * u.cm/u.s).to('km/s'))
ax[0].loglog(dcgs.x / au, (dcgs.vp_g.mean(-1) * u.cm/u.s).to('km/s'))
ax[1].semilogx(dcgs.x / au, (dcgs.vp_g.mean(-1)/dcgs.v_k - 1) * 100)
ax[0].set_ylabel('$v_\phi$ [km/s]')
ax[1].set_ylabel('$\delta v$ [%]')
ax[1].set_xlabel('$r$ [au]');
f.subplots_adjust(hspace=0);
f.savefig('deltav.pdf')

In [ ]:
f, ax = plt.subplots()
ax.semilogx(dcgs.x / au, ((dcgs.vp_g.mean(-1)-dcgs.v_k) * u.cm/u.s).to('km/s'), label=r'$v_\phi-v_k$')
ax.semilogx(dcgs.x / au, (dcgs.vr_g.mean(-1) * u.cm/u.s).to('km/s'), label=r'$v_r$')
ax.set_ylabel('$v$ [%]')
ax.set_xlabel('$r$ [au]')
ax.legend();

### Create $\delta v$ plot like in Teague et al. 2018

In [ ]:
f = twod_plot(dcgs, 10**((dcgs.vp_g / dcgs.v_k[:, None] - 1) * 100), ec='0.3',
              zoom=8, region=region, pos='r',
              #fct='contourf', levels=np.arange(-6,6.5,0.5), extend='both'
              vmin=-6,vmax=6,
              cmap='RdBu_r', r_unit=au, cb_orientation='horizontal')
ax = f.get_axes()[0]
R = 2 * r_pl / au
ax.set_xlim(-R, R)
ax.set_ylim(-R, R)
f.get_axes()[2].set_title('$\delta v$ [%]')
f.get_axes()[2].set_zorder(100)
ax.set_xlabel('$x$ [au]')
ax.set_ylabel('$y$ [au]')
f.savefig('delta_vphi.pdf', **figopts)

In [ ]:
# radial flow
f = twod_plot(dcgs, 10**((dcgs.vr_g*u.cm/u.s).to('km/s').value), ec='0.3',
              zoom=8, region=region, pos='r',
              #fct='contourf', levels=np.linspace(-0.1, 0.1, 10), extend='both',
              vmin=-0.1, vmax=0.1,
              cmap='RdBu_r', r_unit=au, cb_orientation='horizontal')
ax = f.get_axes()[0]
R = 2 * r_pl / au
ax.set_xlim(-R, R)
ax.set_ylim(-R, R)
f.get_axes()[2].set_title('$v_r$ [km/s]')
f.get_axes()[2].set_zorder(100)
ax.set_xlabel('$x$ [au]')
ax.set_ylabel('$y$ [au]')
f.savefig('delta_vr.pdf', **figopts)

### Close the hdf5-file

In [ ]:
f5.close()

# Plots for Akimasa

### Plot Azimuthal size distributions

Load opacities and interpolate at given wavelength and particle size grid

In [ ]:
import dsharp_opac as op

with np.load(op.get_datafile('default_opacities.npz')) as f:
    a = f['a']
    lam = f['lam']
    k_abs = f['k_abs']
    
lam_mm = 0.0870

from scipy.interpolate import interp2d
f_interp = interp2d(np.log10(a), np.log10(lam), np.log10(k_abs).T)
kap = 10.**f_interp(np.log10(dcgs.a), np.log10(lam_mm))

find the maximum outside the planet

In [ ]:
sizes   = [1e-4, 1e-3, 1e-2, 1e-1]
ip      = dcgs.x.searchsorted(r_pl)
sig_d_t = dcgs.sigma_d.sum(-1).mean(-1)
i_max   = ip + sig_d_t[ip:dcgs.x.searchsorted(2 * r_pl)].argmax()
mask    = np.arange(i_max - 5, i_max + 6)

Plot azimuthal distribution in dust surface density

In [ ]:
f, ax = plt.subplots()
ax.semilogy(dcgs.y, dcgs.sigma_g[mask, :].mean(0), label='gas')
ax.semilogy(dcgs.y, dcgs.sigma_d.sum(-1)[mask, :].mean(0), label='total dust')

arr1 = []
arr2 = []

for ia in range(len(sizes)-1):
    ia0 = dcgs.a.searchsorted(sizes[ia])
    ia1 = dcgs.a.searchsorted(sizes[ia + 1])
    
    arr1 += [dcgs.sigma_d[mask, :, ia0:ia1].sum(-1).mean(0)]
    arr2 += [[ia0,ia1]]

    ax.semilogy(dcgs.y, dcgs.sigma_d[mask, :, ia0:ia1].sum(-1).mean(0), label=f'{dcgs.a[ia0]:.2g} ... {dcgs.a[ia1]:.2g}')

ax.set_xlabel('azimuth')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_xticklabels(['0', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3}{2}\, \pi$', r'$2\,\pi$'])
ax.set_xticks([0, np.pi/2, np.pi, 3/2*np.pi, 2*np.pi])
ax.set_xlim(0, 2*np.pi)
ax.legend()
f.savefig('sigma_azi.pdf', transparent=True)

np.savez_compressed('save.npz',{
    'y':dcgs.y,
    'sigma_rmean':np.array(arr1),
    'a0':np.array(arr2)
})

Same plot but opacity weighted

In [ ]:
f, ax = plt.subplots()
ax.semilogy(dcgs.y, dcgs.sigma_g[mask, :].mean(0), label='gas surface density')
ax.semilogy(dcgs.y, (dcgs.sigma_d * kap).sum(-1)[mask, :].mean(0), label='total optical depth')

for ia in range(len(sizes)-1):
    ia0 = dcgs.a.searchsorted(sizes[ia])
    ia1 = dcgs.a.searchsorted(sizes[ia + 1])

    ax.semilogy(dcgs.y, (dcgs.sigma_d * kap)[mask, :, ia0:ia1].sum(-1).mean(0), label=f'{dcgs.a[ia0]:.2g} ... {dcgs.a[ia1]:.2g}')

ax.set_xlabel('azimuth')
ax.set_ylabel(r'$\Sigma$ [g cm$^{-2}$] / $\tau$')
ax.set_xticklabels(['0', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3}{2}\, \pi$', r'$2\,\pi$'])
ax.set_xticks([0, np.pi/2, np.pi, 3/2*np.pi, 2*np.pi])
ax.set_xlim(0, 2*np.pi)
ax.legend()
f.savefig('tau_azi.pdf', transparent=True)

Opacity weighted grain size

In [ ]:
a_bar = (dcgs.a * (1e-40 + dcgs.sigma_d[mask, :, :]) * kap).sum(-1) / ((1e-40 + dcgs.sigma_d[mask, :, :]) * kap).sum(-1)
a_bar = a_bar.mean(0) 

In [ ]:
f, ax = plt.subplots()
ax.semilogy(dcgs.y, dcgs.sigma_g[mask, :].mean(0) / dcgs.sigma_g[mask, :].mean(0).max(), label='gas surface density (normalized)')
ax.semilogy(dcgs.y, a_bar, label=r'$\bar a$')


ax.set_xlabel('azimuth')
ax.set_ylabel(r'$\bar a$ [cm]')
ax.set_xticklabels(['0', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3}{2}\, \pi$', r'$2\,\pi$'])
ax.set_xticks([0, np.pi/2, np.pi, 3/2*np.pi, 2*np.pi])
ax.set_xlim(0, 2*np.pi)
ax.legend()
f.savefig('a_bar.pdf')

In [ ]:
cs2 = dcgs.r * dcgs.v_k* dcgs.params['AspectRatio'] * d.x**-dcgs.params['POWER_ZETA']
T = cs2 / c.k_B.cgs.value * 2.3 * c.m_p.cgs.value
plt.loglog(dcgs.x / au, T)

In [ ]:
bnu = 2 * c.h * nu**3 / c.c**2 /(np.exp(h * nu / (c.k_B * dcgs.T)) - 1)

In [ ]:
tau = (dcgs.sigma_d * kap).sum(-1)

f1 = twod_plot(dcgs, tau, zoom=6, region=[-17, -13, -5, 7], bbox=(1.5, 0.5), r_unit=au, cb_orientation='horizontal', pos='r')
fixplot(f1, f'$\log \\tau_\mathrm{{{lam_mm * 10:.2g}mm}}$')
f1.savefig('tau.pdf', facecolor='none', transparent=True, dpi=300)